In [1]:
from importlib import reload
from datetime import date
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser
import time
from IPython.display import Audio
from cdcqr.common.config import GLASSNODE_API_KEY
sound_file = '/core/tmp/ding.wav'
import sys
sys.path.append('/core/github/cryptoderiv-quant/')
from ct.utils import glassnode
import ct.mlexp2 as ml

2022-01-27 14:03:04,523 - virtualbook - INFO - Table trades created successfully


logging to C:\core\logs\ct.log


In [2]:
ml?

Type:        module
String form: <module 'ct.mlexp2' from 'C:\\core\\github\\cryptoderiv-quant\\ct\\mlexp2.py'>
File:        c:\core\github\cryptoderiv-quant\ct\mlexp2.py
Docstring:   <no docstring>


In [18]:
%matplotlib inline

In [10]:
from cdcqr.data.glassnode import glassnode_data as gnd
gn = gnd.GlassnodeData()

### backtest on Transactions data

In [2]:
df_dict = {}
data_sym = 'BTCUSD_PERP@binance'
df_dict['BTC'] = load_df(data_sym).resample_pv('10Min')

data_sym = 'ETHUSD_PERP@binance'
df_dict['ETH'] = load_df(data_sym).resample_pv('10Min')

In [ ]:
category = 'Transactions'
transactions_features_exchange_activity = ['TransfersVolumeToExchangesSum',
                                             'TransfersVolumeFromExchangesSum',
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersFromExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeFromExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']
transactions_features_exchange_activity1 = ['TransfersVolumeToExchangesSum',                       
                                             'TransfersVolumeExchangesNet',
                                             'TransfersToExchangesCount',
                                             'TransfersVolumeToExchangesMean',
                                             'TransfersVolumeWithinExchangesSum',
                                             'TransfersBetweenExchangesCount',
                                             'TransfersVolumeBetweenExchangesSum']

for feature_name in transactions_features_exchange_activity1:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC', 'ETH']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        df_i = df_dict[underly]
        start_time = df_i.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=underly,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)

In [ ]:
category = 'market'

transactions_features_otc_desks = [ 'mvrv',
 'mvrv_z_score',
 'mvrv_less_155',
 'mvrv_more_155',
]
for feature_name in transactions_features_otc_desks:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC', 'ETH']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        df_i = df_dict[underly]
        start_time = df_i.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=underly,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)

In [ ]:
category = 'OTC'

transactions_features_otc_desks = ['TransfersVolumeToOtcDesksSum', 
                                        'TransfersVolumeFromOtcDesksSum', 
                                        'TransfersToOtcDesksCount', 
                                        'TransfersFromOtcDesksCount', 
                                        ]

for feature_name in transactions_features_otc_desks:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))
        url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
        att_dict = url_parser(url)
        category = att_dict['category']
        feature = att_dict['feature']
        underly = att_dict['a']

        freq='10m'
        df_i = df_dict[underly]
        start_time = df_i.index[0].isoformat()
        url1 = 'metrics~{}~{}'.format(category, feature)
        print(url1)
        dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=underly,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt','vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)
        Audio(sound_file, autoplay=True)

## market - MVRV

In [23]:
category = 'market'

transactions_features_otc_desks = [ 'mvrv',
 'mvrv_z_score',
 'mvrv_less_155',
 'mvrv_more_155',
]

for feature_name in transactions_features_otc_desks:
    print('-----------------------{}---------------------------------------------------------------'.format(feature_name))
    for underly in ['BTC']:
        print('------------------------{}----------------------------------------------------------------'.format(underly))

        fs =  [feature_name]
        a = underly
        i = '1h'
        dfgsi = gn.load_features(fs,a, i)
        dfgsi.columns = ['v']
        df_i = df_i[df_i.index<='20211031']
        dff = dfgsi.reindex(df_i.index)
        #dff['v'] = dff[feature]
        t0 = time.time()
        #display(dff.head())
        res = ml.mlexp2(dff, df_i, sym=underly,
              qsuffix='USDT@binancealt',
              ytrain_thresh=6,
              ytrain_periods=100,
              ytest_thresh=3,
              ytest_periods=100,
              vdelay=2,
              ytypes=["ys"],
              xtypes=["v"],
              qlags=1000,
              request=['vbt', 'vbtplot'],
              #models=["oneiflarger","dt"],
              models=["oneiflarger", 'dt'],
              randcv_n_iter=30,
              sl=2,
              tp=6,
              ts=None,
              vbtfreq="10Min") 
        t1 = time.time()
        print(t1-t0)

-----------------------mvrv---------------------------------------------------------------
------------------------BTC----------------------------------------------------------------
<class 'str'>
loading market mvrv BTC 1h
adding ysbarrier
addysbarrier done 1/2 17.783329248428345
addysbarrier done 2/2 42.73128604888916
['vd']
vd ['vd']
consider using classweight
dummy
{'model': DummyClassifier(strategy='stratified'), 'fixedparams': {}, 'paramsd': {'strategy': ['stratified', 'most_frequent', 'prior', 'uniform', 'constant']}}
oneiflarger
{'model': Classifier1ifLarger(how='any', quantile=0.9999, thresh=1), 'paramsd': {'quantile': [0.8, 0.9, 0.99, 0.999, 0.9999], 'thresh': [0.3, 0.5, 0.9, 1], 'how': ['all', 'any']}, 'fixedparams': {}, 'params': {}}
dt
{'model': DecisionTreeClassifier(class_weight={1: 1}, max_depth=3, max_features=1.0,
                       max_leaf_nodes=5), 'paramsd': {'max_depth': [2, 3], 'min_samples_split': [2], 'min_samples_leaf': [1], 'ccp_alpha': [0.0], 'max_featu